In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [3]:
print(train.shape)
test.shape

(1460, 81)


(1459, 80)

In [4]:
train.head(12)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [5]:
x_train=train.drop(columns="SalePrice",axis=1)
y_train=train["SalePrice"]
x_test=test.copy()

In [6]:
y_train.head(12)

0     208500
1     181500
2     223500
3     140000
4     250000
5     143000
6     307000
7     200000
8     129900
9     118000
10    129500
11    345000
Name: SalePrice, dtype: int64

In [7]:
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

In [8]:
x_train.isnull().sum()

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
BsmtCond           37
BsmtExposure       38
BsmtFinType1       37
BsmtFinSF1          0
BsmtFinType2       38
BsmtFinSF2          0
BsmtUnfSF           0
TotalBsmtSF         0
Heating             0
HeatingQC           0
CentralAir          0
Electrical          1
1stFlrSF            0
2ndFlrSF            0
LowQualFin

In [9]:
x_train_isnull=x_train.isnull().mean()*100

In [10]:
x_train_ex_null=x_train.drop(columns=x_train_isnull[x_train_isnull>20].keys(),axis=1)

In [11]:
x_train_ex_null.isnull().sum().sum()

868

In [12]:
x_train_ex_null_num=x_train_ex_null.select_dtypes(include=["int64","float64"])
x_train_ex_null_obj=x_train_ex_null.select_dtypes(include=["O"])

In [13]:
x_train_ex_null_num.isnull().sum()

Id                 0
MSSubClass         0
LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea         8
BsmtFinSF1         0
BsmtFinSF2         0
BsmtUnfSF          0
TotalBsmtSF        0
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       0
BsmtHalfBath       0
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt       81
GarageCars         0
GarageArea         0
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
dtype: int64

In [14]:
x_train_null_lst=[var for var in x_train_ex_null_num if x_train_isnull[var]>0]

In [15]:
x_train_null_lst

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [16]:
x_train_ex_null_num[x_train_null_lst].isnull().sum()

LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64

In [17]:
x_train_null_obj_lst=[var for var in x_train_ex_null_obj if x_train_isnull[var]>0]

In [18]:
x_train_null_obj_lst

['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [19]:
x_train_mean=["LotFrontage"]
x_train_median=["MasVnrArea","GarageYrBlt"]
x_train_mode_obj=['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2']
x_train_constant_obj=['Electrical',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [20]:
x_train_mean_imputer=Pipeline(steps=[("imputer",SimpleImputer())])
x_train_median_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="median"))])
x_train_mode_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="most_frequent"))])
x_train_constant_imputer=Pipeline(steps=[("imputer",SimpleImputer(strategy="constant",fill_value="missing"))])

In [21]:
x_train_mean_imputer.steps[0]

('imputer', SimpleImputer())

In [22]:
preprocessor=ColumnTransformer(transformers=[("mean imputer",x_train_mean_imputer,x_train_mean),
                               ("median imputer",x_train_median_imputer,x_train_median),
                               ("mode imputer",x_train_mode_imputer,x_train_mode_obj),
                               ("constant imputer",x_train_constant_imputer,x_train_constant_obj)])


In [23]:
preprocessor.fit(x_train)

ColumnTransformer(transformers=[('mean imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2']),
                                ('constant imputer',
                                 Pipeline(steps=[('imputer',
                                               

In [24]:
preprocessor.transform

<bound method ColumnTransformer.transform of ColumnTransformer(transformers=[('mean imputer',
                                 Pipeline(steps=[('imputer', SimpleImputer())]),
                                 ['LotFrontage']),
                                ('median imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['MasVnrArea', 'GarageYrBlt']),
                                ('mode imputer',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent'))]),
                                 ['MasVnrType', 'BsmtQual', 'BsmtCond',
                                  'BsmtExposure', 'BsmtFinType1',
                                  'BsmtFinType2']),
                                ('constant imputer',
                                 Pipeline(steps=[('imputer',
  

In [25]:
preprocessor.named_transformers_["mean imputer"].named_steps["imputer"].statistics_

array([70.04995837])

In [26]:
preprocessor.named_transformers_["median imputer"].named_steps["imputer"].statistics_

array([   0., 1980.])

In [27]:
train["MasVnrArea"].mean()

103.68526170798899

In [28]:
train["GarageYrBlt"].mean()

1978.5061638868744